# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]},
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is buy
Number of combinations is 75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [05:38<00:00,  4.51s/it]


# Check local statistics

In [2]:
# 15m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
69,STOCH_RSI,16,30,9,7,3,30,95.68,78.42,86.33,86.33,86.33,89.21,87.77,90.65,90.65,91.37,91.37,90.65,87.77,87.77,86.33,86.33,88.49,87.05,87.77,87.77,81.29,81.29,78.42,68.35,1.12,1.57,2.21,2.61,3.25,3.88,3.93,4.12,4.26,3.77,3.74,3.88,3.99,3.84,4.23,4.16,4.80,4.96,5.28,5.27,4.97,4.94,2.80,2.10,139,86.391250,3.736667,2278.383750,519.396667
68,STOCH_RSI,16,30,9,7,3,25,95.65,78.26,86.23,86.23,86.23,89.13,87.68,90.58,90.58,91.30,91.30,90.58,87.68,87.68,86.23,86.23,88.41,86.96,87.68,87.68,81.16,81.16,78.26,68.12,1.13,1.51,2.21,2.63,3.24,3.91,3.96,4.09,4.30,3.77,3.73,3.82,4.02,3.85,4.30,4.20,4.84,4.99,5.30,5.29,5.01,4.95,2.86,2.12,138,86.291667,3.751250,2248.250000,517.672500
18,STOCH_RSI,12,30,9,7,3,25,95.56,82.78,87.78,87.78,88.33,90.00,88.33,90.56,90.56,90.56,89.44,89.44,86.11,86.11,85.56,86.11,88.33,87.22,86.11,85.00,79.44,80.00,76.67,70.56,1.15,1.82,2.41,2.84,3.51,4.13,4.05,4.39,4.35,3.77,3.73,3.75,3.86,3.77,4.10,4.33,4.84,4.98,5.22,5.14,5.01,4.85,2.92,2.42,180,86.180833,3.805833,2912.550000,685.050000
67,STOCH_RSI,16,30,9,7,3,20,95.35,78.29,87.60,86.05,86.82,89.15,87.60,89.92,89.92,91.47,90.70,89.92,87.60,87.60,86.82,86.05,88.37,86.82,87.60,87.60,80.62,80.62,77.52,67.44,1.10,1.43,2.21,2.59,3.22,3.93,3.93,4.12,4.33,3.74,3.71,3.75,3.98,3.84,4.23,4.15,4.57,4.96,5.28,5.01,4.96,4.80,2.80,2.03,129,86.143750,3.694583,2082.543750,476.601250
17,STOCH_RSI,12,30,9,7,3,20,95.27,83.43,88.76,87.57,88.76,89.94,88.17,89.94,89.94,90.53,88.76,88.76,85.80,85.80,85.80,85.80,88.17,86.98,85.80,84.62,78.70,79.29,75.74,70.41,1.12,1.74,2.40,2.80,3.51,4.15,4.02,4.42,4.37,3.77,3.74,3.75,3.78,3.73,3.99,4.16,4.67,4.96,5.16,5.00,4.96,4.64,2.91,2.14,169,85.947500,3.745417,2695.127500,632.975417
19,STOCH_RSI,12,30,9,7,3,30,95.06,88.89,87.65,86.42,88.89,88.89,86.42,87.65,87.65,85.19,82.72,85.19,80.25,82.72,81.48,82.72,82.72,82.72,82.72,85.19,86.42,85.19,86.42,87.65,0.95,1.82,2.10,2.89,3.36,4.37,4.02,4.17,4.09,3.61,3.41,3.59,3.78,4.46,5.26,6.30,6.30,6.68,6.50,6.99,7.69,8.16,8.83,8.70,81,85.700833,4.917917,1271.767500,398.351250
44,STOCH_RSI,14,30,9,7,3,30,95.60,81.13,86.16,86.16,86.79,88.68,86.79,89.31,89.31,89.31,89.31,88.68,86.16,86.16,84.91,84.91,86.79,85.53,85.53,86.16,80.50,80.50,77.36,71.70,1.10,1.64,2.24,2.71,3.31,3.93,3.85,4.17,4.26,3.74,3.71,3.76,3.98,3.84,4.23,4.36,4.80,4.96,5.31,5.31,5.22,4.94,2.92,2.31,159,85.560000,3.775000,2474.040000,600.225000
43,STOCH_RSI,14,30,9,7,3,25,95.57,81.01,86.08,86.08,86.71,88.61,86.71,89.24,89.24,89.24,89.24,88.61,86.08,86.08,84.81,84.81,86.71,85.44,85.44,86.08,80.38,80.38,77.22,71.52,1.11,1.63,2.22,2.73,3.28,3.95,3.89,4.15,4.30,3.72,3.69,3.75,3.88,3.85,4.30,4.38,4.84,4.98,5.35,5.32,5.23,4.95,2.93,2.42,158,85.470417,3.785417,2444.325833,598.095833
42,STOCH_RSI,14,30,9,7,3,20,95.24,80.95,87.07,85.71,87.07,88.44,86.39,88.44,88.44,89.12,88.44,87.76,85.71,85.71,85.03,84.35,86.39,85.03,85.03,85.71,79.59,79.59,76.

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [12], 'low_bound': [30]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [20]},
                'LinearReg': {'timeperiod': [6], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            